<a href="https://colab.research.google.com/github/omkar806/on-device-experiments/blob/main/knowledge_base_graphQL_code_generation_completed_chat_flow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Run the below cell to install all the libraries

In [1]:
!pip install -q langchain openai langchain_openai langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.1/324.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 10.2 MB/s eta 0:00:00


#Import all the neccessary libraries using the below cell

In [17]:
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI
from langchain.callbacks import get_openai_callback
import openai
import os
from google.colab import userdata
from collections import deque
import requests
import json

#Defining a Pydantic Model to extract the exact information for Each key and Validate the json output given by the LLM.

#Initialising the OpenAI model , Please provide your valid OPENAI_API_KEY in the cell below in the first line

In [3]:
API_KEY=userdata.get("OPENAI_API_KEY")
model_name = "gpt-3.5-turbo-instruct"
temperature = 0.0
model = OpenAI(openai_api_key=API_KEY,model_name=model_name, temperature=temperature, max_tokens=800)

#Function responsible for taking the user query processing it and Validating it with Pydantic model keys and printing the Json output

In [4]:
graphQL_template = """

The response should understand a user question and return a graphql query according to it.
The Supabase table under consideration is named as "receipt_radar_insights" and it has the following column_names:
purchaseCategories,competitorReceipts,shoppingDates,spendingCapacity
The response must ONLY contain the code snippet and NOTHING else.
The response must be one single line which contains only the query and must not be assigned to a variable.
Below are some examples for you to what response you have to exactly generate.
Example 1:
Question: what's is previuous month budget ?
Response:
{
  receipt_radar_insightsCollection
{
  edges
{
  node
{shoppingDates spendingCapacity}

}
  }
    }

Example 2:
Question: What did I spend on last month?
Response:
{
  receipt_radar_insightsCollection
{
  edges
{
  node
{purchaseCategories}

}
  }
    }

"""

In [24]:
project_ref = userdata.get("PROJECT_REF")
api_key = userdata.get("SUPABASE_API_KEY")
url = f"https://{project_ref}.supabase.co/graphql/v1"
headers = {
    "apiKey": api_key,
    "Content-Type": "application/json"
}

query = """{
  receipt_radar_insightsCollection
{
  edges
{
  node
{spendingCapacity}

}
  }
}"""

payload = {
    "query": query,
}

response = requests.post(url, headers=headers, json=payload)

print(response.json())

{'data': {'receipt_radar_insightsCollection': {'edges': [{'node': {'spendingCapacity': 8686.375}}]}}}


In [33]:
def get_user_data_supabase(supabase_query):
  project_ref = os.getenv("PROJECT_REF")
  api_key = os.getenv("SUPABASE_API_KEY")

  # Define the URL and headers
  url = f"https://{project_ref}.supabase.co/graphql/v1"
  headers = {
      "apiKey": api_key,
      "Content-Type": "application/json"
  }

  # Define the GraphQL query
  query = f"{supabase_query}"

  print(query)
  print(type(query))
  # Define the payload
  payload = {
      "query": query,

  }

  # Make the POST request
  response = requests.post(url, headers=headers, json=payload)

  # Print the response
  print(response.json())
  return response.json()

In [35]:
def bot_code_generation_response(user_query):
  try:
    prompt = PromptTemplate(
      template="Answer the user query based on the below examples and template.\n{graphQL_template}\n User Query :\n {query} \n Generate a graphql query based on above question and examples given.",
      input_variables=["query"],
      partial_variables={"graphQL_template": graphQL_template},
    )
    input = prompt.format_prompt(query=str(user_query))
    # print(input.to_string())
    with get_openai_callback() as cb:
      try:
        result = model.invoke(input.to_string())
        response=get_user_data_supabase(result)
        print("printing response in bot_code_generation")
        print("printing result",result)
        return response
        # return result
      except Exception as e:
        print(f"Error {e}")
        return e
  except Exception as e:
    print(f"Error : {e}")
    return e


#Below cell is the response of the bot on the user_query

In [21]:
#defining a prompt template for
user_query_response_template = """
Answer the user query based on the context provided.\n

User query : {query} \n

context : {context}\n

Response History : {response_history}\n
Above is the context and response history provided ,context is a json analyse it and answer the user query above based on this context and the user_history ,if the question is related to any historical reponses.

  """

In [36]:
def bot_response_supabase(user_query,user_data,response_history):
  try:
    prompt = PromptTemplate(
      template=user_query_response_template,
      input_variables=["query"],
      partial_variables={"context": user_data , "response_history":str(list(response_history))},
    )
    input = prompt.format_prompt(query=str(user_query))
    # print(input.to_string())
    with get_openai_callback() as cb:
      try:
        result = model.invoke(input.to_string())
        print("printing result")
        print(result)
      except Exception as e:
        print(f"Error {e}")

      try:
        response_history.append(result)
      except Exception as e:
        response_history.append("No valid previous response")
        print(f"Error {e}")
  except Exception as e:
    print(f"Error : {e}")


In [37]:
message_history = deque(maxlen=4)
while True:
  user_query = input("User: ")
  user_query = user_query
  if user_query == "exit":
    break
  user_data=json.dumps(bot_code_generation_response(user_query))
  #generating a proper response based on user query
  bot_response_supabase(user_query,user_data,message_history)


User: what is my budget ? 


{
  receipt_radar_insightsCollection
{
  edges
{
  node
{spendingCapacity}

}
  }
}
<class 'str'>
Error HTTPSConnectionPool(host='none.supabase.co', port=443): Max retries exceeded with url: /graphql/v1 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7f4eac903a00>: Failed to resolve 'none.supabase.co' ([Errno -2] Name or service not known)"))


TypeError: Object of type ConnectionError is not JSON serializable